# 11.3.7 Implementing the QR Factorization

With this notebook, you will implement the QR factorization of a matrix $ A $ with linearly independent columns, producing matrices $ Q $ and $ R $ such that $ A = Q R $.  The algorithm is equivalent to performing Gram-Schmidt orthogonalization on the vectors that are the columns of $ A $.

<font color=red> Be sure to make a copy!!!! </font>

<h2> First, let's create matrices $ A $, $ Q $, and $ R $.  </h2>

In [1]:
import numpy as np
import laff
import flame

A = np.matrix( ' 1., -1.,  2;\
                 2.,  1., -3;\
                -1.,  3.,  2;\
                 0., -2., -1' )

Q = np.matrix( np.zeros( (4,3) ) )

R = np.matrix( np.zeros( (3,3) ) )

print( 'A = ' )
print( A )


A = 
[[ 1. -1.  2.]
 [ 2.  1. -3.]
 [-1.  3.  2.]
 [ 0. -2. -1.]]


In [2]:
type(A)

numpy.matrix

In [3]:
len(A.shape) 

2

<h2> Implement the QR factorization algorithm from 11.3.7 </h2>

Here is the algorithm:

<img src="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/11_3_7_QR_factorization.png" alt="QR factorization algorithm" width=75%>
    
<font color=red> Important: if you make a mistake, rerun ALL cells above the cell in which you were working, and then the one where you are working. </font>

Create the routine
<code> QR_Gram_Schmidt_unb</code>
with the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> for the algorithm

Hints:
<ul>
<li>
You will want to store $ a_1^\perp $ in <code> q1 </code>.  This will mean first copying <code> a1 </code> to <code> q1 </code>.
</li>
<li>
The following routines will be useful:
<ul>
<li>
<code> laff.copy ( x, y ) </code>
</li>
<li>
<code> laff.gemv ( trans, alpha, A, x, beta, y ) </code>
</li>
<li>
<code> laff.norm2 ( x ) </code> <br>
The annoying thing is that this returns the value.  You will want do to <br>
<code> rho11[:,:] = laff.norm2 </code>.
</li>
<li>
<code> laff.invscal ( alpha, x ) </code>
</li>
</ul>
</li>
</ul>


In [4]:
import flame
import laff as laff

def QR_Gram_Schmidt_unb(A, Q, R):

    AL, AR = flame.part_1x2(A, \
                            0, 'LEFT')

    QL, QR = flame.part_1x2(Q, \
                            0, 'LEFT')

    RTL, RTR, \
    RBL, RBR  = flame.part_2x2(R, \
                               0, 0, 'TL')

    while AL.shape[1] < A.shape[1]:

        A0, a1, A2 = flame.repart_1x2_to_1x3(AL, AR, \
                                             1, 'RIGHT')

        Q0, q1, Q2 = flame.repart_1x2_to_1x3(QL, QR, \
                                             1, 'RIGHT')

        R00,  r01,   R02,  \
        r10t, rho11, r12t, \
        R20,  r21,   R22   = flame.repart_2x2_to_3x3(RTL, RTR, \
                                                     RBL, RBR, \
                                                     1, 1, 'BR')

        #------------------------------------------------------------#

        laff.gemv( 'Transpose', 1.0, Q0, a1, 0.0, r01 )
        laff.copy( a1, q1 )
        laff.gemv( 'No transpose', -1.0, Q0, r01, 1.0, q1 )
        rho11[:,:] = laff.norm2( q1 )
        laff.invscal( rho11, q1 )

        #------------------------------------------------------------#

        AL, AR = flame.cont_with_1x3_to_1x2(A0, a1, A2, \
                                            'LEFT')

        QL, QR = flame.cont_with_1x3_to_1x2(Q0, q1, Q2, \
                                            'LEFT')

        RTL, RTR, \
        RBL, RBR  = flame.cont_with_3x3_to_2x2(R00,  r01,   R02,  \
                                               r10t, rho11, r12t, \
                                               R20,  r21,   R22,  \
                                               'TL')

    flame.merge_1x2(QL, QR, Q)

    flame.merge_2x2(RTL, RTR, \
                    RBL, RBR, R)



<h3> Test the routine </h3>

<font color=red> Important: if you make a mistake, rerun ALL cells above the cell in which you were working, and then the one where you are working. </font>

In [5]:
QR_Gram_Schmidt_unb( A, Q, R )

print( 'A = ' )
print( A )

print( 'Q = ' )
print( Q )

print( 'R = ' )
print( R )

print( 'Q * R - A:' )
print( Q * R - A )

A = 
[[ 1. -1.  2.]
 [ 2.  1. -3.]
 [-1.  3.  2.]
 [ 0. -2. -1.]]
Q = 
[[ 0.40824829 -0.17609018  0.8820199 ]
 [ 0.81649658  0.44022545 -0.32318287]
 [-0.40824829  0.70436073  0.23565417]
 [ 0.         -0.52827054 -0.24912013]]
R = 
[[ 2.44948974 -0.81649658 -2.44948974]
 [ 0.          3.7859389   0.26413527]
 [ 0.          0.          3.45401687]]
Q * R - A:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


The result should be a 4 x 3 (approximately) zero matrix.

Now, let's check if the columns of $ Q $ are mutually orthogonal:

In [6]:
print( np.transpose( Q ) * Q )

[[1.00000000e+00 9.85167115e-17 1.66533454e-16]
 [9.85167115e-17 1.00000000e+00 3.64291930e-17]
 [1.66533454e-16 3.64291930e-17 1.00000000e+00]]


The above should approximately be a 3 x 3 identity matrix.

## Related to the enrichment.

If you read the enrichment on the Gram-Schmidt method, you will find the following interesting...

In [7]:
import numpy as np
import laff
import flame

epsilon = 1.0e-7

A = np.matrix( ' 1.,       1.,      1.;\
                 0,        0.,      0.;\
                 0.,       0,       0.;\
                 0.,       0.,      0.' )

A[ 1,0 ] = epsilon
A[ 2,1 ] = epsilon
A[ 3,2 ] = epsilon

#  This creates the matrix
# A = np.matrix( ' 1.,       1.,      1.;\
#                  epsilon,  0.,      0.;\
#                  0.,       epsilon, 0.;\
#                  0.,       0.,      epsilon' )

Q = np.matrix( np.zeros( (4,3) ) )

R = np.matrix( np.zeros( (3,3) ) )

print( 'A = ' )
print( A )


A = 
[[1.e+00 1.e+00 1.e+00]
 [1.e-07 0.e+00 0.e+00]
 [0.e+00 1.e-07 0.e+00]
 [0.e+00 0.e+00 1.e-07]]


In [8]:
QR_Gram_Schmidt_unb( A, Q, R )

print( 'A = ' )
print( A )

print( 'Q = ' )
print( Q )

print( 'R = ' )
print( R )

print( 'Q * R - A:' )
print( Q * R - A )

A = 
[[1.e+00 1.e+00 1.e+00]
 [1.e-07 0.e+00 0.e+00]
 [0.e+00 1.e-07 0.e+00]
 [0.e+00 0.e+00 1.e-07]]
Q = 
[[ 1.00000000e+00  6.90840682e-08  4.07886015e-08]
 [ 1.00000000e-07 -7.07106781e-01 -4.17602698e-01]
 [ 0.00000000e+00  7.07106781e-01 -3.98821881e-01]
 [ 0.00000000e+00  0.00000000e+00  8.16424579e-01]]
R = 
[[1.00000000e+00 1.00000000e+00 1.00000000e+00]
 [0.00000000e+00 1.41421356e-07 6.90840682e-08]
 [0.00000000e+00 0.00000000e+00 1.22485288e-07]]
Q * R - A:
[[0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.81806312e-25]
 [0.00000000e+00 0.00000000e+00 3.28376723e-24]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]]


This should equal, approximately, the zero matrix.

Now, let's check if the columns of Q are mutually orthogonal:

In [9]:
print( np.transpose( Q ) * Q )

[[ 1.00000000e+00 -1.62660994e-09 -9.71668373e-10]
 [-1.62660994e-09  1.00000000e+00  1.32800433e-02]
 [-9.71668373e-10  1.32800433e-02  1.00000000e+00]]


You will notice that the resulting 3 x 3 matrix, which should (approximately) equal the identity matrix, has off-diagonal entries that do not equal zero at all.  

One of them even equals 0.5 (when I tried)